In [1]:
import pandas as pd
import urllib
from urllib import request
import numpy as np
import sklearn 
from statistics import mean 
import statistics
from sklearn import preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn import neural_network
from sklearn.neural_network import MLPRegressor
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split 
from subprocess import check_output
from datetime import time
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import feature_selection

C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

In [3]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['time']
del df['datenew']
#SETTING DATE COLUMN AS INDEX
df = df.set_index('date')

In [4]:
df.head(10)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,...,18,11,0,66000,0,0,0,0,0,0


In [5]:
def mae(org,pred):
    return mean_absolute_error(org,pred)
def rmse(org,pred):
    return np.sqrt(mean_squared_error(org,pred))
def r2s(org,pred):
    return r2_score(org,pred)
def mape(org,pred):
    y_true, y_pred = np.array(org), np.array(pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
def scores(model,yorg,ypred):
    print("The scores for the model : ",model)
    print("MAE    :    ",mae(yorg,ypred))
    print("RMSE   :    ",rmse(yorg,ypred))
    print("R2     :    ",r2s(yorg,ypred))
    print("MAPE   :    ",mape(yorg,ypred))

In [7]:
data1 = df
data1 = data1.drop(['Appliances'],axis=1)

In [8]:
data1

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,40,19.890000,46.026667,19.200000,44.500000,19.790000,44.933333,18.890000,45.730000,17.133333,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,50,19.890000,45.766667,19.200000,44.500000,19.790000,44.900000,18.890000,45.790000,17.100000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,50,19.856667,45.560000,19.200000,44.500000,19.730000,44.900000,18.890000,45.863333,17.100000,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,40,19.790000,45.597500,19.200000,44.433333,19.730000,44.790000,18.890000,45.790000,17.166667,...,18,11,0,66000,0,0,0,0,0,0


In [9]:
X = data1
y = df.Appliances



In [ ]:
#X = X.values


In [ ]:
#y = y.values

In [10]:
# splitting X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7,
                                                    random_state=1)

In [ ]:
#est= MLPRegressor()
#minmax_scaler = preprocessing.MinMaxScaler()
#selector = feature_selection.RFE(est)
#pipe_params = [('feat_selection',selector),('std_scaler', minmax_scaler), ('clf', est)]
##pipe = Pipeline(pipe_params)

In [ ]:
# Declare hyperparameters to tune
#param_grid = {'activation' : ['tanh', 'relu'],'solver' : ['adam', 'sgd'] }
#param_grid = {'solver' : ['adam', 'sgd'] }


In [ ]:
#clf1 = GridSearchCV(pipe, param_grid=param_grid, cv=10)


In [ ]:
#clf1.fit(X_train, y_train)

In [ ]:
param_grid1 = {'activation' : ['tanh', 'relu'],'solver' : ['adam', 'sgd'],
              'learning_rate_init': [0.001,0.1],'alpha':[0.0001,0.15]}

In [ ]:
param_grid1

In [12]:
param_grid2 = {'activation' : ['tanh', 'relu'],'solver' : ['adam', 'sgd'],'alpha':[0.0001,0.15]}

In [13]:
param_grid2

{'activation': ['tanh', 'relu'],
 'solver': ['adam', 'sgd'],
 'alpha': [0.0001, 0.15]}

In [14]:
rf = MLPRegressor()

In [15]:
gs = GridSearchCV(rf, param_grid=param_grid2, cv = 3, n_jobs = -1, verbose=3, refit=True)

In [16]:
gs.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:   15.8s remaining:    5.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   18.1s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'activation': ['tanh', 'relu'], 'solver': ['adam', 'sgd'], 'alpha': [0.0001, 0.15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [17]:
gs.best_params_

{'activation': 'relu', 'alpha': 0.0001, 'solver': 'adam'}

In [18]:
mlp = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto',learning_rate_init=0.001)


In [19]:
X_train

,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-05-09 10:40:00,0,24.600000,39.290000,27.700000,32.826667,26.200000,37.230000,24.790000,38.000000,23.100000,...,10,9,0,38400,0,0,0,0,0,0
2016-04-19 01:20:00,0,21.290000,37.230000,18.700000,39.260000,22.600000,36.500000,22.000000,34.290000,20.133333,...,1,19,0,4800,0,0,0,0,1,0
2016-04-28 08:30:00,0,20.600000,36.430000,19.860000,37.100000,21.426667,34.433333,20.066667,35.560000,18.730000,...,8,28,0,30600,0,0,0,1,0,0
2016-05-05 07:30:00,0,22.290000,36.090000,19.666667,37.700000,23.290000,35.826667,22.290000,32.860000,20.790000,...,7,5,0,27000,0,0,0,1,0,0
2016-05-03 23:50:00,0,22.700000,35.172500,20.260000,36.700000,23.000000,36.363333,22.500000,33.700000,21.100000,...,23,3,0,85800,0,0,0,0,1,0
2016-04-20 14:50:00,0,22.700000,32.590000,22.890000,30.100000,22.390000,35.500000,22.790000,32.795714,19.790000,...,14,20,0,53400,0,0,0,0,0,1
2016-05-27 03:50:00,0,24.033333,44.933333,22.390000,45.290000,26.700000,40.290000,24.500000,42.200000,22.390000,...,3,27,0,13800,1,0,0,0,0,0
2016-04-13 19:40:00,0,22.200000,44.060000,20.290000,45.726667,24.323333,41.400000,22.290000,41.223333,20.903750,...,19,13,0,70800,0,0,0,0,0,1
2016-05-19 01:00:00,0,23.390000,41.400000,21.080000,43.700000,24.600000,39.560000,23.200000,39.590000,21.100000,...,1,19,0,3600,0,0,0,1,0,0


In [20]:
xtrain = X_train.values

In [21]:
y_train

date
2016-05-09 10:40:00     50
2016-04-19 01:20:00     30
2016-04-28 08:30:00     70
2016-05-05 07:30:00     70
2016-05-03 23:50:00     40
2016-04-20 14:50:00    100
2016-05-27 03:50:00     60
2016-04-13 19:40:00    210
2016-05-19 01:00:00     40
2016-03-27 13:20:00     70
2016-02-07 03:30:00     40
2016-05-03 21:20:00     50
2016-05-14 09:30:00    100
2016-04-03 14:00:00     90
2016-03-11 17:10:00     40
2016-03-25 14:00:00    250
2016-02-14 00:40:00     60
2016-03-05 02:20:00     50
2016-02-25 16:50:00     90
2016-05-10 23:40:00     70
2016-04-28 01:20:00     50
2016-05-10 23:00:00     60
2016-04-30 11:40:00     70
2016-03-02 23:00:00     50
2016-04-08 03:50:00     50
2016-03-04 07:10:00     20
2016-04-08 16:20:00    330
2016-01-14 19:30:00    100
2016-01-19 04:40:00     30
2016-04-01 17:50:00     50
                      ... 
2016-01-29 11:50:00     60
2016-03-17 21:10:00    110
2016-02-05 18:00:00     90
2016-05-24 11:10:00    100
2016-02-11 08:50:00    410
2016-01-18 20:50:00    

In [22]:
ytrain = y_train.values

In [23]:
mlp.fit(xtrain,ytrain)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [24]:
xtest = X_test.values
ytest = y_test.values

In [25]:
ypredtest_unscaled = mlp.predict(xtest)

In [26]:
ypredtest_unscaled

array([ 46.25905253, 143.39573059, 134.85885559, ..., 146.49021976,
       141.16597385, 144.67343943])

In [27]:
print("Scores for test")
scores("MLP - Unscaled",y_test,ypredtest_unscaled)

Scores for test
The scores for the model :  MLP - Unscaled
MAE    :     61.67816318000781
RMSE   :     97.59679285883654
R2     :     0.09857145930683764
MAPE   :     77.08913651755677


In [29]:
ypredtrain_unscaled = mlp.predict(xtrain)

In [30]:
print("Scores for train")
scores("MLP - Unscaledcaled",y_train,ypredtrain_unscaled)

Scores for train
The scores for the model :  MLP - Unscaledcaled
MAE    :     61.48895382030703
RMSE   :     98.11750059737194
R2     :     0.08190884556741285
MAPE   :     78.98945398293198



## Scaling the data and tuning hyperparameters

In [31]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [32]:

xscaled_train = scaler.fit_transform(X_train)

In [33]:
xscaled_test = scaler.fit_transform(X_test)

In [34]:
xscaled_train

array([[0.        , 0.82470961, 0.40034813, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.47518479, 0.33311575, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.40232313, 0.30700609, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.49736008, 0.23716275, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.56071806, 0.54960836, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.21119324, 0.55602698, ..., 0.        , 0.        ,
        1.        ]])

In [35]:
xscaled_test

array([[0.        , 0.22949666, 0.40916667, ..., 1.        , 0.        ,
        0.        ],
       [0.4       , 0.61351637, 0.45388889, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.83526927, 0.37180556, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.2       , 0.43294615, 0.20916667, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.53854277, 0.32324074, ..., 0.        , 0.        ,
        0.        ],
       [0.2       , 0.42238648, 0.41305556, ..., 0.        , 0.        ,
        1.        ]])

In [36]:
ytestscaled= y_test.values
ytestscaled = ytestscaled.reshape(len(ytestscaled),1)
yscaled_test = scaler.fit_transform(ytestscaled)

C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [37]:
yscaled_test 

array([[0.04878049],
       [0.06097561],
       [0.1097561 ],
       ...,
       [0.09756098],
       [0.7195122 ],
       [0.17073171]])

In [40]:
ytrainscaled= y_train.values
ytrainscaled = ytrainscaled.reshape(len(ytrainscaled),1)
yscaled_train = scaler.fit_transform(ytrainscaled)

C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [52]:
yscaled_train

array([[0.03738318],
       [0.01869159],
       [0.05607477],
       ...,
       [0.07476636],
       [0.04672897],
       [0.01869159]])

In [55]:
yscaled_train1 =yscaled_train.ravel()


In [56]:
yscaled_train1

array([0.03738318, 0.01869159, 0.05607477, ..., 0.07476636, 0.04672897,
       0.01869159])

In [57]:
gs.fit(xscaled_train, yscaled_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  18 out of  24 | elapsed:   15.2s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   17.0s finished
C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'activation': ['tanh', 'relu'], 'solver': ['adam', 'sgd'], 'alpha': [0.0001, 0.15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [58]:
gs.best_params_

{'activation': 'relu', 'alpha': 0.15, 'solver': 'adam'}

In [59]:
gs.best_estimator_

MLPRegressor(activation='relu', alpha=0.15, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [60]:
mlpscaled = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.15, batch_size='auto',  max_iter=200,learning_rate_init=0.01)


In [61]:
mlpscaled.fit(xscaled_train, yscaled_train)


C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.15, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.01, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [62]:
ypredtestcvss = mlpscaled.predict(xscaled_test)

In [63]:
print("Scores for test")
scores("MLP - scaled optimal ",yscaled_test,ypredtestcvss)

Scores for test
The scores for the model :  MLP - scaled optimal 
MAE    :     0.059845147712305105
RMSE   :     0.11662458862318652
R2     :     0.13449782041181857


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


MAPE   :     inf


In [64]:
ypredtraincvss = mlpscaled.predict(xscaled_train)

In [65]:
print("Scores for train")
scores("MLP - scaled optimal ",yscaled_train,ypredtraincvss)

Scores for train
The scores for the model :  MLP - scaled optimal 
MAE    :     0.05726702567409587
RMSE   :     0.08880402291211191
R2     :     0.13895532434624946


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


MAPE   :     inf
